In [2]:
import asyncio
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.genai import types

# Import your local modules
from veganflow_ai.agents.procurement import create_procurement_agent
from veganflow_ai.tools.retail_database_setup import setup_retail_database

# --- 1. Setup Infrastructure ---
print("📦 Resetting Retail Database...")
setup_retail_database()

print("🧠 Initializing Memory Service...")
memory_service = InMemoryMemoryService()
session_service = InMemorySessionService()

# --- 2. Seed Strategy for TWO Products ---
print("🌱 Seeding Strategic Targets into Memory...")

seeder_agent = LlmAgent(name="seeder", model=Gemini(model="gemini-2.0-flash"))
seed_id = "seed_session"
user_id = "notebook_user"

# FIX: Use Keyword Arguments for create_session
await session_service.create_session(
    app_name="seeder_app", 
    user_id=user_id, 
    session_id=seed_id
)

# Define strategy for Oat Milk AND Truffle Brie
strategies = [
    "Strategy: For 'Oat Barista Blend', target price is $3.30. Max $3.50.",
    "Strategy: For 'Cultured Truffle Brie', target price is $9.20. It is a luxury item, so prioritize quality/reliability over speed."
]

seeder_runner = Runner(
    agent=seeder_agent, 
    session_service=session_service, 
    app_name="seeder_app"
)

# Inject strategies
for strat in strategies:
    async for _ in seeder_runner.run_async(
        user_id=user_id, 
        session_id=seed_id, 
        new_message=types.Content(parts=[types.Part(text=strat)])
    ):
        pass

# FIX: Use Keyword Arguments for get_session
seed_session = await session_service.get_session(
    app_name="seeder_app", 
    user_id=user_id, 
    session_id=seed_id
)
await memory_service.add_session_to_memory(seed_session)
print("✅ Strategies for Oat Milk and Cheese ingested.")

# --- 3. Initialize Procurement Agent ---
print("🤖 Spawning Procurement Negotiator...")
agent = create_procurement_agent()

# Create a session for the actual tests
test_session_id = "negotiation_test_session"

# FIX: Use Keyword Arguments for create_session
await session_service.create_session(
    app_name="procurement_test", 
    user_id=user_id, 
    session_id=test_session_id
)

runner = Runner(
    agent=agent,
    session_service=session_service,
    memory_service=memory_service,
    app_name="procurement_test"
)

# --- 4. TEST CASE 1: Oat Milk (Commodity) ---
print("\n" + "="*50)
print("🧪 TEST 1: Negotiating for 'Oat Barista Blend'")
print("="*50)

query_1 = "Find vendors for 'Oat Barista Blend' and negotiate a purchase of 100 units."
await runner.run_debug(query_1)

# --- 5. TEST CASE 2: Truffle Brie (Artisanal) ---
print("\n" + "="*50)
print("🧪 TEST 2: Negotiating for 'Cultured Truffle Brie'")
print("="*50)

# This should trigger a completely different vendor (e.g., Rebel Cheese or The Vreamery)
query_2 = "Now find vendors for 'Cultured Truffle Brie'. We need 20 units. Get the best deal."
await runner.run_debug(query_2)

App name mismatch detected. The runner is configured with app name "seeder_app", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".


📦 Resetting Retail Database...
✅ Database 'veganflow_store.db' rebuilt with 21 products and 16 competing offers.
   - CRITICAL SCENARIO: Oat Barista Blend has 0.8 days supply.
🧠 Initializing Memory Service...
🌱 Seeding Strategic Targets into Memory...


App name mismatch detected. The runner is configured with app name "procurement_test", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".


✅ Strategies for Oat Milk and Cheese ingested.
🤖 Spawning Procurement Negotiator...

🧪 TEST 1: Negotiating for 'Oat Barista Blend'

 ### Created new session: debug_session_id

User > Find vendors for 'Oat Barista Blend' and negotiate a purchase of 100 units.


/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/veganflow_ai/agents/procurement.py:70: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_agent = RemoteA2aAgent(
App name mismatch detected. The runner is configured with app name "procurement_negotiation_task", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".


...

App name mismatch detected. The runner is configured with app name "procurement_negotiation_task", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".



🔄 [A2A] Initiating Handshake with http://localhost:8006...
📨 [A2A] Sending Offer: 20x Cultured Truffle Brie @ $8.55...
Tb [A2A] Vendor Replied: I am unable to accept this offer. The lowest I can go is $10.08.



App name mismatch detected. The runner is configured with app name "procurement_negotiation_task", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".



🔄 [A2A] Initiating Handshake with http://localhost:8006...
📨 [A2A] Sending Offer: 20x Cultured Truffle Brie @ $10.08...
Tb [A2A] Vendor Replied: I accept your offer. I can supply 20 units of Cultured Truffle Brie at $10.08. Delivery in 2 days.

procurement_negotiator > Deal secured with Rebel Cheese! After a bit of negotiation, we've locked in 20 units of 'Cultured Truffle Brie' at $10.08 each. This was a better deal than the next available vendor.


[Event(model_version='gemini-2.5-pro', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'product_name': 'Cultured Truffle Brie'
         },
         id='adk-9b899c59-b52c-49cd-aa18-303146637e1d',
         name='get_vendor_options'
       ),
       thought_signature=b'\n\xf2\x04\x01\xe3\xf1\xff^\x99\x7f*\x01\xbf\x8e \xdf\xac\xbd\x1a\xaa\x84Z\x0b\x1c\x1eU\xb1\x92k\\\x98l\x1di$v\x96\xc0\xcda,\xf9\x06R\x9c\x7fP\n\xaeT\xdb\xe8=/v\x13S\xe3\x99\xfe\xb2\x136M\xa3\x8b\x82\xcb\xf3\x80\x83\xff^\r\nr#0\xe99\x0c\xe9;\xff\xf7\xeb\xb8\xb1\xee\xa7a\x82|\xc0LQ\xb8...'
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=13,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: '